# Estalações e download da database



In [26]:
!wget -O listings.csv.gz "https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2025-03-19/data/listings.csv.gz"

!gunzip -k listings.csv.gz

--2025-09-24 16:44:07--  https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2025-03-19/data/listings.csv.gz
Resolving data.insideairbnb.com (data.insideairbnb.com)... 18.239.18.9, 18.239.18.42, 18.239.18.19, ...
Connecting to data.insideairbnb.com (data.insideairbnb.com)|18.239.18.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21857120 (21M) [application/x-gzip]
Saving to: ‘listings.csv.gz’

listings.csv.gz     100%[===================>]  20.84M  --.-KB/s    in 0.07s   

2025-09-24 16:44:08 (284 MB/s) - ‘listings.csv.gz’ saved [21857120/21857120]

gzip: listings.csv already exists; do you wish to overwrite (y or n)? y


In [27]:
!pip install gradio matplotlib seaborn pandas scikit-learn lazypredict


# **Bibliotecas**

In [28]:
import gradio as gr
import seaborn as sns
import pandas as pd
from os import name
import matplotlib.pyplot as plt
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from lazypredict.Supervised import LazyRegressor
from torch.utils.data import TensorDataset, DataLoader, Subset, random_split
import torch.nn as nn
import torch.optim as optim
%matplotlib inline
plt.style.use('fivethirtyeight')
import numpy as np
import datetime
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split

In [29]:
pd.set_option('display.max_columns', None)

df = pd.read_csv("listings.csv")
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2302715,https://www.airbnb.com/rooms/2302715,20250319150537,2025-03-20,city scrape,guilherme,"House with five large bedrooms, living room, g...",NaN,https://a0.muscache.com/pictures/30842883/b19c...,11706874,https://www.airbnb.com/users/show/11706874,Guilherme,2014-01-27,"Rio, Brazil",NaN,NaN,NaN,NaN,f,https://a0.muscache.com/im/users/11706874/prof...,https://a0.muscache.com/im/users/11706874/prof...,NaN,2.00,2.00,"['email', 'phone']",t,f,NaN,Anil,NaN,-22.95,-43.33,Entire home,Entire home/apt,12,5.00,5 baths,5.00,10.00,"[""Washer"", ""Hot tub"", ""Breakfast"", ""TV with st...","$2,700.00",10,30,10,10,30,30,10.00,30.00,NaN,t,30,60,90,365,2025-03-20,0,0,0,287,0,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
1,2304688,https://www.airbnb.com/rooms/2304688,20250319150537,2025-03-20,city scrape,OLIMPÍADAS 2016,"Apartment with living room, 2 bedrooms and kit...","Beaches, restaurants, supermarkets, amusement ...",https://a0.muscache.com/pictures/30867360/fe14...,10405816,https://www.airbnb.com/users/show/10405816,Eurico,2013-12-03,"Rio de Janeiro, Brazil",NaN,NaN,NaN,0%,f,https://a0.muscache.com/im/users/10405816/prof...,https://a0.muscache.com/im/users/10405816/prof...,NaN,1.00,1.00,"['email', 'phone']",t,f,"Rio, Rio de Janeiro, Brazil",Ipanema,NaN,-22.99,-43.19,Entire rental unit,Entire home/apt,4,1.00,1 bath,2.00,3.00,"[""Washer"", ""TV with standard cable"", ""Wifi"", ""...",$837.00,7,14,7,7,14,14,7.00,14.00,NaN,t,30,60,90,365,2025-03-20,0,0,0,287,0,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
2,2306547,https://www.airbnb.com/rooms/2306547,20250319150537,2025-03-20,city scrape,Amplo Apto com 3 Dormitórios em Copacabana - 1...,Excellent apartment in Copacabana. Cozy and la...,The Copacabana Neighborhood is independent an...,https://a0.muscache.com/pictures/4923e99d-7190...,4107346,https://www.airbnb.com/users/show/4107346,Vera Lucia,2012-11-09,"Rio de Janeiro, Brazil",Sou uma pessoa que adora receber turistas no R...,within a few hours,100%,62%,f,https://a0.muscache.com/im/users/4107346/profi...,https://a0.muscache.com/im/users/4107346/profi...,Copacabana,1.00,1.00,"['email', 'phone']",t,t,"Copacabana, Rio de Janeiro, Brazil",Copacabana,NaN,-22.97,-43.19,Entire rental unit,Entire home/apt,7,2.00,2 baths,3.00,6.00,"[""Dedicated workspace"", ""Dishes and silverware...",$800.00,5,90,5,5,90,90,5.00,90.00,NaN,t,25,47,77,257,2025-03-20,13,3,1,257,4,30,24000.00,2020-01-07,2025-03-05,5.00,5.00,4.92,5.00,4.92,5.00,4.77,NaN,f,1,1,0,0,0.21
3,2306767,https://www.airbnb.com/rooms/2306767,20250319150537,2025-03-20,city scrape,S

In [30]:
target_columns = [
    "accommodates",      # Number of guests the property can host
    "bathrooms",         # Number of bathrooms available
    "bedrooms",          # Number of bedrooms available
    "beds",              # Number of beds available
    "minimum_nights",    # Minimum nights required for a booking
    "maximum_nights",    # Maximum nights allowed for a booking
    "number_of_reviews", # Total reviews given by past guests
    "neighbourhood_cleansed", #
    "property_type", # How a guest books space
    "amenities", # Basic items
    "review_scores_rating", #
    "estimated_occupancy_l365d",
    "price"              # Nightly rental price
]

rio_listings = df[target_columns].copy()

rio_listings.sample(5)

,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,number_of_reviews,neighbourhood_cleansed,property_type,amenities,review_scores_rating,estimated_occupancy_l365d,price
6132,5,2.00,2.00,4.00,2,90,92,Barra da Tijuca,Entire condo,"[""Shared beach access \u2013 Beachfront"", ""Win...",4.75,96,$532.00
39334,4,1.00,1.00,2.00,2,365,3,Santo Cristo,Entire rental unit,"[""Dedicated workspace"", ""Dishes and silverware...",5.00,18,$262.00
23683,2,1.00,1.00,2.00,2,30,7,Ipanema,Entire rental unit,"[""Dedicated workspace"", ""Beach access \u2013 B...",5.00,18,$468.00
23779,3,1.00,0.00,2.00,2,365,24,Copacabana,Entire rental unit,"[""Beach access \u2013 Beachfront"", ""Baking she...",4.88,138,$400.00
22311,2,1.00,1.00,1.00,4,365,3,Copacabana,Entire rental unit,"[""Dedicated workspace"", ""Cleaning available du...",5.00,8,$325.00


In [31]:
rio_listings.describe()

,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,estimated_occupancy_l365d
count,42013.00,38313.00,41373.00,38335.00,42013.00,42013.00,42013.00,33140.00,42013.00
mean,3.96,1.50,1.52,2.36,3.61,366.08,23.68,4.80,49.72
std,2.27,0.97,1.02,2.12,14.25,379.76,45.88,0.39,69.01
min,1.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,2.00,1.00,1.00,1.00,1.00,45.00,1.00,4.76,0.00
50%,4.00,1.00,1.00,2.00,2.00,365.00,6.00,4.92,18.00
75%,5.00,2.00,2.00,3.00,3.00,365.00,26.00,5.00,72.00
max,16.00,20.00,30.00,56.00,1000.00,1125.00,726.00,5.00,255.00


In [32]:
rio_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42013 entries, 0 to 42012
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   accommodates               42013 non-null  int64  
 1   bathrooms                  38313 non-null  float64
 2   bedrooms                   41373 non-null  float64
 3   beds                       38335 non-null  float64
 4   minimum_nights             42013 non-null  int64  
 5   maximum_nights             42013 non-null  int64  
 6   number_of_reviews          42013 non-null  int64  
 7   neighbourhood_cleansed     42013 non-null  object 
 8   property_type              42013 non-null  object 
 9   amenities                  42013 non-null  object 
 10  review_scores_rating       33140 non-null  float64
 11  estimated_occupancy_l365d  42013 non-null  int64  
 12  price                      38356 non-null  object 
dtypes: float64(4), int64(5), object(4)
memory usag

tratamento da coluna preço

In [33]:
stripped_commas = rio_listings['price'].str.replace(',', '')

stripped_dollars = stripped_commas.str.replace('$', '')

rio_listings['price'] = stripped_dollars.astype('float')

In [34]:
rio_listings.dropna(axis=0, inplace=True)

In [35]:
ordem = rio_listings['property_type'].astype('category').unique().tolist()
rio_listings['property_type_cod'] = pd.Categorical(rio_listings['property_type'], categories=ordem, ordered=True)

rio_listings['property_type_cod'] = rio_listings['property_type_cod'].cat.codes

In [36]:
ordem = rio_listings['neighbourhood_cleansed'].astype('category').unique().tolist()
rio_listings['neighbourhood_cleansed_cod'] = pd.Categorical(rio_listings['neighbourhood_cleansed'], categories=ordem, ordered=True)

rio_listings['neighbourhood_cleansed_cod'] = rio_listings['neighbourhood_cleansed_cod'].cat.codes

In [37]:
ordem = rio_listings['amenities'].astype('category').unique().tolist()
rio_listings['amenities_cod'] = pd.Categorical(rio_listings['amenities'], categories=ordem, ordered=True)

rio_listings['amenities_cod'] = rio_listings['amenities_cod'].cat.codes

In [38]:
rio_listings.sample(5)

,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,number_of_reviews,neighbourhood_cleansed,property_type,amenities,review_scores_rating,estimated_occupancy_l365d,price,property_type_cod,neighbourhood_cleansed_cod,amenities_cod
34718,6,1.50,2.00,3.00,4,30,7,Copacabana,Entire rental unit,"[""Beach access \u2013 Beachfront"", ""Dishes and...",5.00,56,380.00,0,0,25338
34724,2,1.00,1.00,1.00,1,60,18,Centro,Entire rental unit,"[""Dedicated workspace"", ""Hot water"", ""Cleaning...",4.72,108,140.00,0,13,25343
25269,4,1.00,1.00,2.00,2,365,33,Leblon,Entire rental unit,"[""Dedicated workspace"", ""Cleaning available du...",4.82,174,657.00,0,7,19066
24933,7,2.50,3.00,4.00,2,7,6,Barra de Guaratiba,Entire home,"[""Baking sheet"", ""Canal view"", ""Clothing stora...",5.00,24,204.00,7,35,18833
19196,2,1.50,1.00,1.00,2,31,60,Lagoa,Entire rental unit,"[""Dedicated workspace"", ""Cleaning available du...",5.00,228,657.00,0,12,14627


In [39]:
rio_listings = rio_listings.drop(['neighbourhood_cleansed', 'property_type', 'amenities'], axis=1)

In [40]:
rio_listings

,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,estimated_occupancy_l365d,price,property_type_cod,neighbourhood_cleansed_cod,amenities_cod
2,7,2.00,3.00,6.00,5,90,13,5.00,30,800.00,0,0,0
3,3,1.00,0.00,2.00,3,90,193,4.69,114,190.00,0,0,1
4,4,1.00,1.00,2.00,2,28,28,4.89,24,220.00,0,0,2
5,2,1.00,1.00,1.00,2,730,36,4.94,48,401.00,1,1,3
6,3,1.00,0.00,2.00,7,730,58,4.79,140,219.00,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41883,4,1.00,1.00,1.00,1,365,3,5.00,18,275.00,0,0,28711
41907,4,1.00,2.00,2.00,1,365,1,5.00,6,329.00,0,22,28712
41913,2,1.00,1.00,1.00,1,365,2,5.00,12,180.00,0,41,28713
41943,6,1.00,2.00,3.00,1,365,1,5.00,6,405.00,0,0,25249


In [41]:
class Architecture:
  def __init__(self, model, loss_fn, optimizer):
      self.model = model
      self.loss_fn = loss_fn
      self.optimizer = optimizer
      self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
      self.model.to(self.device)
      self.train_loader = None
      self.val_loader = None
      self.losses = []
      self.val_losses = []
      self.total_epochs = 0
      self.train_step_fn = self._make_train_step_fn()
      self.val_step_fn = self._make_val_step_fn()

  def set_loaders(self, train_loader, val_loader=None):
      self.train_loader = train_loader
      self.val_loader = val_loader

  def _make_train_step_fn(self):
      def fn(x, y):
          self.model.train()
          yhat = self.model(x)
          loss = self.loss_fn(yhat, y)
          loss.backward()
          self.optimizer.step()
          self.optimizer.zero_grad()
          return loss.item()
      return fn

  def _make_val_step_fn(self):
      def fn(x, y):
          self.model.eval()
          yhat = self.model(x)
          loss = self.loss_fn(yhat, y)
          return loss.item()
      return fn

  def _mini_batch(self, validation=False):
      loader = self.val_loader if validation else self.train_loader
      step_fn = self.val_step_fn if validation else self.train_step_fn
      losses = []
      for x, y in loader:
          x, y = x.to(self.device), y.to(self.device)
          losses.append(step_fn(x, y))
      return np.mean(losses)

  def set_seed(self, seed=42):
      torch.manual_seed(seed)
      np.random.seed(seed)

  def train(self, n_epochs, seed=42):
      self.set_seed(seed)
      for _ in range(n_epochs):
          self.total_epochs += 1
          self.losses.append(self._mini_batch(False))
          with torch.no_grad():
              self.val_losses.append(self._mini_batch(True))

  def plot_losses(self):
      fig, ax = plt.subplots(figsize=(10, 4))

      if self.losses and self.val_losses:
          ax.plot(self.losses, label='Training Loss', color='blue')
          ax.plot(self.val_losses, label='Validation Loss', color='red')
          ax.set_yscale('log')
      else:
          ax.text(0.5, 0.5, 'Sem dados de perda disponíveis', ha='center', va='center')

      ax.set_xlabel('Epochs')
      ax.set_ylabel('Loss')
      ax.legend()
      fig.tight_layout()
      return fig


In [42]:
from matplotlib.figure import Figure

In [45]:
import gradio as gr
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split, Subset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# ==============================================================================
# Carregamento e Limpeza Inicial dos Dados (coloque seu código de carregar dados aqui)
# ==============================================================================
# Supondo que 'rio_listings' já foi carregado e pré-processado como no seu notebook
# Exemplo de como os dados deveriam estar antes deste bloco:
# !wget -q -O listings.csv.gz "https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2025-03-19/data/listings.csv.gz"
# rio_listings_raw = pd.read_csv("listings.csv.gz", compression='gzip')
# target_columns = [
#     'accommodates', 'bathrooms', 'bedrooms', 'beds', 'property_type',
#     'neighbourhood_cleansed', 'number_of_reviews', 'review_scores_rating',
#     'cancellation_policy', 'host_is_superhost', 'amenities', 'price'
# ]
# rio_listings = rio_listings_raw[target_columns].copy()
# rio_listings['price'] = rio_listings['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
# rio_listings.dropna(subset=[c for c in target_columns if c != 'amenities'], inplace=True)
# for col in ['property_type', 'neighbourhood_cleansed', 'cancellation_policy', 'host_is_superhost', 'amenities']:
#      rio_listings[col] = rio_listings[col].astype('category').cat.codes
# ==============================================================================


# Funções de outliers (código inalterado)
def _iqr_bounds(series: pd.Series, k: float = 1.5):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    low = q1 - k * iqr
    up = q3 + k * iqr
    return low, up, {"Q1": q1, "Q3": q3, "IQR": iqr}

def remove_outliers_iqr(df, columns, k=1.5, inclusive=True, dropna=True):
    data = df.copy()
    for c in columns:
        data[c] = pd.to_numeric(data[c], errors="coerce")
    if dropna:
        data = data.dropna(subset=columns)
    before = len(data)
    bounds = {}
    mask = pd.Series(True, index=data.index)
    for c in columns:
        low, up, stats = _iqr_bounds(data[c].dropna(), k=k)
        bounds[c] = {"low": low, "up": up, **stats}
        if inclusive:
            m = (data[c] >= low) & (data[c] <= up)
        else:
            m = (data[c] > low) & (data[c] < up)
        mask &= m
    cleaned = data.loc[mask].copy()
    info = {
        "rows_in": before,
        "rows_out": len(cleaned),
        "rows_removed": before - len(cleaned),
        "k": k,
        "inclusive": inclusive,
        "bounds": bounds,
    }
    return cleaned, info

# Aplica limpeza (simulação, pois 'rio_listings' não está definido aqui)
# Em um notebook real, esta célula seria executada
try:
    rio_iqr_input = rio_listings.select_dtypes(include="number").copy()
    rio_iqr, summary = remove_outliers_iqr(rio_iqr_input, list(rio_iqr_input.columns))
except NameError:
    # Cria um dataframe de exemplo caso o original não exista
    print("Aviso: 'rio_listings' não encontrado. Usando dados de exemplo.")
    data_exemplo = {
        'accommodates': np.random.randint(1, 10, 1000), 'bathrooms': np.random.uniform(1, 5, 1000),
        'bedrooms': np.random.uniform(1, 6, 1000), 'beds': np.random.randint(1, 10, 1000),
        'number_of_reviews': np.random.randint(0, 200, 1000), 'review_scores_rating': np.random.uniform(1, 5, 1000),
        'price': np.random.uniform(100, 2000, 1000)
    }
    rio_listings_exemplo = pd.DataFrame(data_exemplo)
    # Adicionando alguns outliers de propósito
    rio_listings_exemplo.loc[990:, 'price'] = 10000
    rio_iqr, summary = remove_outliers_iqr(rio_listings_exemplo, list(rio_listings_exemplo.columns))


# Classe Architecture (código inalterado)
class Architecture(object):
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_loader = None
        self.val_loader = None
        self.losses = []
        self.val_losses = []

    def set_loaders(self, train_loader, val_loader):
        self.train_loader = train_loader
        self.val_loader = val_loader

    def train_step(self, x, y):
        self.model.train()
        yhat = self.model(x)
        loss = self.loss_fn(yhat, y)
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        return loss.item()

    def train(self, n_epochs=1):
        self.losses = []
        self.val_losses = []
        for epoch in range(n_epochs):
            batch_losses = []
            for x_batch, y_batch in self.train_loader:
                loss = self.train_step(x_batch, y_batch)
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in self.val_loader:
                    self.model.eval()
                    yhat = self.model(x_val)
                    val_loss = self.loss_fn(yhat, y_val).item()
                    batch_val_losses.append(val_loss)
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)
        return self

    def plot_losses(self):
        fig = plt.figure(figsize=(10, 4))
        plt.plot(self.losses, label='Training Loss', c='b')
        plt.plot(self.val_losses, label='Validation Loss', c='r')
        plt.yscale('log')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('Training and Validation Losses')
        plt.grid(True)
        plt.close(fig)
        return fig

    def predict(self, x):
        self.model.eval()
        x_tensor = torch.as_tensor(x).float()
        with torch.no_grad():
            y_hat = self.model(x_tensor)
        return y_hat.numpy()


# ==============================================================================
# Funções para as abas da Interface Gráfica
# ==============================================================================

def aba_preprocessamento():
    explicacao = """
    ## Pré-processamento: O Alicerce do Modelo

    Antes de qualquer análise, os dados brutos precisam ser preparados. Esta etapa é crucial para o sucesso do modelo.

    ### 1. Tratamento de Variáveis Categóricas (`Label Encoding`)
    Modelos de machine learning entendem números, não textos como "Copacabana". A técnica usada aqui foi o **Label Encoding**, que atribui um número único para cada categoria (ex: Copacabana -> 0, Ipanema -> 1).

    **Por que não usamos *Dummies (One-Hot Encoding)*, que é teoricamente mais correto?**
    - A abordagem com *dummies* cria uma nova coluna para cada categoria. Para variáveis como `amenities`, com milhares de combinações, isso geraria um dataset com milhares de colunas.
    - Este fenômeno, conhecido como **Maldição da Dimensionalidade**, tornaria o treinamento computacionalmente caro e exigiria muito mais dados para evitar overfitting.
    - Portanto, o `Label Encoding` foi uma escolha **pragmática** para manter o modelo gerenciável.

    ### 2. Normalização dos Dados (`Z-Score`)
    As variáveis numéricas possuem escalas muito diferentes (ex: `número de reviews` vai de 0 a 500, enquanto `banheiros` vai de 1 a 5). A normalização **Z-Score** coloca todas as variáveis em uma escala semelhante (média 0 e desvio padrão 1).

    **Por que não usar outras técnicas como *Min-Max* ou *Log-Transformation*?**
    - **Min-Max Scaler:** Re-escala os dados para um intervalo [0, 1]. Embora suavize a curva de aprendizado visualmente, é **extremamente sensível a outliers**. Um único valor extremo pode "achatar" todos os outros dados em um intervalo minúsculo, dificultando o aprendizado do modelo.
    - **Log Transformation:** É útil para dados com distribuição muito assimétrica. No nosso caso, após a remoção de outliers, os dados já estão mais bem distribuídos. Aplicar um log aqui poderia **distorcer a relação linear** que o modelo tenta aprender, gerando mais ruído do que sinal.
    """
    return explicacao

def aba_outliers():
    texto = f"**Rows before**: {summary['rows_in']}  \n"
    texto += f"**Rows after**: {summary['rows_out']}  \n"
    texto += f"**Removed**: {summary['rows_removed']}  \n\n"
    texto += "**Per-column bounds (IQR method):** \n"
    for col, b in summary["bounds"].items():
        texto += f"- `{col}`: [{b['low']:.3f}, {b['up']:.3f}] (Q1={b['Q1']:.3f}, Q3={b['Q3']:.3f}, IQR={b['IQR']:.3f})  \n"

    explicacao = """
    ### Remoção de Outliers
    Nesta etapa utilizamos o **método do IQR (Intervalo Interquartil)** para detectar e remover valores
    extremos (outliers) das variáveis numéricas.

    **Por que isso é importante?**
    - Outliers podem distorcer a média, a variância e as correlações.
    - Eles podem "puxar" o aprendizado do modelo na direção errada, fazendo com que ele tenha um desempenho ruim nos casos mais comuns.
    - Após a remoção, temos um conjunto de dados mais **limpo** e **representativo**, essencial para um treinamento robusto.
    """
    return texto + "\n\n" + explicacao

def aba_heatmap():
    corr_matrix = rio_iqr.corr()
    fig = plt.figure(figsize=(20, 10))
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", center=0, square=True, cbar_kws={"shrink": 0.75})
    plt.title("Correlation Heatmap of Rio Listings Features", fontsize=16, pad=15)
    plt.xticks(rotation=45, ha="right")
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.close(fig)

    texto = """
    ### Heatmap de Correlação
    O mapa de calor mostra visualmente como cada variável se relaciona com as demais.
    - **Valores próximos de +1 (vermelho quente):** Indicam forte correlação positiva. Quando uma variável aumenta, a outra tende a aumentar também.
    - **Valores próximos de -1 (azul frio):** Indicam forte correlação negativa. Quando uma aumenta, a outra tende a diminuir.
    - **Valores próximos de 0 (branco):** Indicam pouca ou nenhuma relação linear.
    """
    return fig, texto

def aba_descricao_price():
    tabela = rio_iqr["price"].describe().to_frame().to_markdown()
    texto = """
    ### Estatísticas Descritivas de `price`
    Aqui temos as principais medidas estatísticas para a nossa variável-alvo, o preço.
    - **count**: Quantidade de imóveis no dataset após a limpeza.
    - **mean**: O preço médio dos aluguéis.
    - **std**: O desvio padrão, que indica a variabilidade ou "espalhamento" dos preços em torno da média.
    - **min / max**: O preço do imóvel mais barato e do mais caro.
    - **25%, 50% (mediana), 75%**: Os quartis, que nos ajudam a entender a distribuição dos preços. 50% dos imóveis custam menos que o valor da mediana.
    """
    return tabela + "\n\n" + texto

def aba_correlacao_price():
    price_corr = rio_iqr.corr()["price"].sort_values(ascending=False)
    texto = "**Correlação de cada variável com `price`:** \n"
    for col, val in price_corr.items():
        texto += f"- `{col}`: {val:.3f}  \n"

    explicacao = """
    ### O que mais influencia o preço?
    Esta lista ordenada é uma das análises mais importantes. Ela mostra o "poder de influência" de cada característica sobre o preço final.
    - **Correlações positivas altas** (ex: `accommodates`, `bedrooms`): São os fatores que mais contribuem para aumentar o preço.
    - **Correlações fracas (próximas de 0)**: Sugerem pouca influência direta sobre o preço.
    """
    return texto + "\n\n" + explicacao

def aba_treinamento(n_epochs):
    feature_cols = [c for c in rio_iqr.columns if c != "price"]
    target_col = "price"
    X = rio_iqr[feature_cols].to_numpy(dtype=np.float32)
    y = rio_iqr[target_col].to_numpy(dtype=np.float32).reshape(-1, 1)

    x_tensor = torch.as_tensor(X).float()
    y_tensor = torch.as_tensor(y).float()
    dataset = TensorDataset(x_tensor, y_tensor)

    ratio = 0.8
    n_total = len(dataset)
    n_train = int(n_total * ratio)
    train_data, val_data = random_split(dataset, [int(n_total * ratio), n_total - n_train])
    train_idx, val_idx = train_data.indices, val_data.indices

    eps = 1e-8
    mu = x_tensor[train_idx].mean(dim=0)
    std = torch.where(x_tensor[train_idx].std(dim=0, unbiased=False) < eps, torch.ones_like(mu), x_tensor[train_idx].std(dim=0, unbiased=False))
    y_mu = y_tensor[train_idx].mean(dim=0)
    y_std = torch.where(y_tensor[train_idx].std(dim=0, unbiased=False) < eps, torch.ones_like(y_mu), y_tensor[train_idx].std(dim=0, unbiased=False))

    x_tensor_z = (x_tensor - mu) / std
    y_tensor_z = (y_tensor - y_mu) / y_std

    dataset_z = TensorDataset(x_tensor_z, y_tensor_z)
    train_data, val_data = Subset(dataset_z, train_idx), Subset(dataset_z, val_idx)
    train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
    val_loader = DataLoader(dataset=val_data, batch_size=16)

    D = x_tensor.shape[1]
    model = nn.Sequential(nn.Linear(D, 1))
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()
    arch = Architecture(model, loss_fn, optimizer)
    arch.set_loaders(train_loader, val_loader)
    arch.train(n_epochs=n_epochs)
    fig = arch.plot_losses()

    texto = f"""
    ### Treinamento do Modelo: Ensinando a Máquina

    Aqui construímos e treinamos nosso modelo de **Regressão Linear** usando PyTorch.

    #### O que são Épocas (`Epochs`)?
    Uma época representa uma "passagem completa" do modelo por todos os dados de treinamento. Com o slider, você pode controlar "quantas vezes o modelo estuda o material".
    - **Cuidado:** Mais épocas não é sempre melhor! Se a perda de validação (vermelho) começar a subir enquanto a de treino (azul) continua caindo, temos **overfitting**. Isso significa que o modelo está "decorando" os dados de treino em vez de aprender a generalizar para novos dados.

    #### Por que Regressão Linear?
    É o modelo mais fundamental. Ele tenta encontrar a melhor relação linear (uma "linha" ou "plano") entre as características do imóvel e seu preço. A fórmula que ele aprende é:
    `Preço = (peso1 * feature1) + (peso2 * feature2) + ... + bias`

    #### Por que o otimizador SGD e não o Adam?
    O otimizador é o algoritmo que ajusta os "pesos" do modelo para minimizar o erro.
    - **SGD (Stochastic Gradient Descent):** É como um caminhante cauteloso descendo uma montanha. É simples, robusto e, para um problema "simples" como a regressão linear (cuja superfície de erro é uma "tigela perfeita"), ele é perfeitamente adequado e tende a encontrar soluções que generalizam bem.
    - **Adam:** É como um carro de corrida inteligente, muito mais rápido e complexo. É excelente para problemas difíceis (redes neurais profundas), mas para este caso, sua complexidade é desnecessária e o SGD é a escolha mais sólida e didática.
    """
    return fig, texto


def aba_lazy():
    X = rio_iqr.drop("price", axis=1)
    y = rio_iqr["price"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    linear_reg_model = LinearRegression()
    linear_reg_model.fit(X_train, y_train)
    y_pred = linear_reg_model.predict(X_test)

    fig = plt.figure(figsize=(10, 6))
    sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2, label='Linha Perfeita (y=x)')
    plt.title('Preços Reais vs. Preços Previstos (Regressão Linear)', fontsize=16)
    plt.xlabel('Preço Real')
    plt.ylabel('Preço Previsto')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.close(fig)

    texto = """
    ### PyTorch vs. Ferramentas Automatizadas (LazyPredict)

    Esta aba demonstra o resultado de um modelo de Regressão Linear simples (da biblioteca Scikit-learn, similar ao que o LazyPredict faria). O gráfico compara o preço real com a previsão.

    #### Qual a diferença de filosofia?
    - **PyTorch (usado na aba de Treinamento):** É uma **"oficina de engenharia"**. Te dá controle total para construir, customizar e entender cada detalhe do modelo. É ideal para aprender, pesquisar e ter máxima flexibilidade. Construir uma regressão linear aqui é como montar um motor peça por peça para entender como ele funciona.
    - **LazyPredict / Scikit-learn:** São como um **"showroom de carros"**. Permitem testar dezenas de modelos prontos rapidamente para ver qual se encaixa melhor no problema, sem focar nos detalhes internos. É ideal para obter um bom resultado rápido.

    **Conclusão:** Para o objetivo deste projeto (aprender e controlar o processo), a abordagem com PyTorch é mais rica. Para um projeto focado em velocidade de entrega, começar com uma ferramenta como LazyPredict para "mapear o terreno" seria mais eficiente.
    """
    return fig, texto


# ==============================================================================
# Construção da Interface
# ==============================================================================

demo = gr.TabbedInterface(
    interface_list=[
        gr.Interface(fn=aba_preprocessamento, inputs=[], outputs=gr.Markdown(), title="Pré-processamento e Normalização"),
        gr.Interface(fn=aba_outliers, inputs=[], outputs=gr.Markdown(), title="Remoção de Outliers"),
        gr.Interface(fn=aba_heatmap, inputs=[], outputs=[gr.Plot(), gr.Markdown()], title="Heatmap de Correlação"),
        gr.Interface(fn=aba_descricao_price, inputs=[], outputs=gr.Markdown(), title="Estatísticas de Preço"),
        gr.Interface(fn=aba_correlacao_price, inputs=[], outputs=gr.Markdown(), title="Correlação com Preço"),
        gr.Interface(
            fn=aba_treinamento,
            inputs=gr.Slider(minimum=1, maximum=150, value=50, step=1, label="Número de Épocas"),
            outputs=[gr.Plot(), gr.Markdown()],
            title="🧠 Treinamento com PyTorch"
        ),
        gr.Interface(fn=aba_lazy, inputs=[], outputs=[gr.Plot(), gr.Markdown()], title="⚙️ Análise do Modelo Final")
    ],
    tab_names=[
        "🛠️ Pré-processamento",
        "🔍 Remoção de Outliers",
        "🌡️ Heatmap",
        "📊 Estatísticas",
        "🔗 Correlações",
        "🧠 Treinamento",
        "⚙️ Análise Final"
    ],
    title="Análise Preditiva de Preços do AirBnB no Rio de Janeiro"
)

# Para executar em um notebook, use a linha abaixo
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6ecc602c500f8b7f52.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
